# **Case 2 – CDS Stripping**
> **Credit, Complexity and Systemic Risk**

In [1]:
# Imports –––––
import pandas as pd
import numpy as np
from scipy.optimize import brentq

## **Data setup**

In [2]:
# Market Data & Assumptions –––––

# Loss Given Default
fLGD = 0.40

# Risk-free rate
fRISK_FREE = 0.03

# CDS maturities (years)
vMATURITIES = np.array([1, 3, 5, 7, 10])

# CDS spreads (basis points to decimal)
vCDS_BPS = np.array([100, 110, 120, 120, 125])
vCDS = vCDS_BPS / 10000

## **The Simple Model**

In [3]:
# Hazard Rate Estimation –––––

# Average Hazard Rates
vAVG_HAZARD_RATES_Q1 = vCDS / fLGD

# Forward Hazard Rates
vFWD_HAZARD_RATES_Q1 = np.zeros(len(vMATURITIES))
vFWD_HAZARD_RATES_Q1[0] = vAVG_HAZARD_RATES_Q1[0]
for i in range(1, len(vMATURITIES)):
    numerator = vAVG_HAZARD_RATES_Q1[i] * vMATURITIES[i] - vAVG_HAZARD_RATES_Q1[i-1] * vMATURITIES[i-1]
    vFWD_HAZARD_RATES_Q1[i] = numerator / (vMATURITIES[i] - vMATURITIES[i-1])

# Default Probabilities
vSURVIVAL_PROBS_Q1 = np.exp(-vAVG_HAZARD_RATES_Q1 * vMATURITIES)
vDEFAULT_PROBS_Q1 = np.zeros(len(vMATURITIES))
vDEFAULT_PROBS_Q1[0] = 1 - vSURVIVAL_PROBS_Q1[0]
for i in range(1, len(vMATURITIES)):
    vDEFAULT_PROBS_Q1[i] = vSURVIVAL_PROBS_Q1[i-1] - vSURVIVAL_PROBS_Q1[i]

dfRESULTS_Q1 = pd.DataFrame({
    'Maturity': [f'{m}Y' for m in vMATURITIES],
    'CDS Rate (bps)': vCDS_BPS,
    '(Average) Hazard Rate': vAVG_HAZARD_RATES_Q1,
    'Forward Hazard Rate': vFWD_HAZARD_RATES_Q1,
    'Default Probability': vDEFAULT_PROBS_Q1
})
print("\n", dfRESULTS_Q1.to_string(index=False))


 Maturity  CDS Rate (bps)  (Average) Hazard Rate  Forward Hazard Rate  Default Probability
      1Y             100                0.02500             0.025000             0.024690
      3Y             110                0.02750             0.028750             0.054498
      5Y             120                0.03000             0.033750             0.060103
      7Y             120                0.03000             0.030000             0.050124
     10Y             125                0.03125             0.034167             0.078969


In [4]:
# Export dfRESULTS_Q1 to LaTeX –––––
with open('Documentation/Tables/ccsr-a2-q1-results.tex', 'w') as f:
    f.write(dfRESULTS_Q1.to_latex(index=False, float_format="%.6f"))

## **The Exact Model**

In [5]:
# CDS Pricing Function –––––

# Survival probability function based on piecewise constant hazard rates
def survival_probability(t, lambda_periods):
    integral = 0.0
    for t_start, t_end, lam in lambda_periods:
        if t <= t_start:
            break
        elif t <= t_end:
            integral += lam * (t - t_start)
            break
        else:
            integral += lam * (t_end - t_start)
    return np.exp(-integral)

def price_cds(maturity, cds_rate, lgd, r, lambda_periods):
    num_quarters = int(maturity * 4)
    payment_times = np.linspace(0.25, maturity, num_quarters)
    
    premium_leg = 0.0
    for i, t_i in enumerate(payment_times):
        t_prev = 0 if i == 0 else payment_times[i-1]
        dt = t_i - t_prev
        q_surv = survival_probability(t_i, lambda_periods)
        df = np.exp(-r * t_i)
        premium_leg += cds_rate * df * dt * q_surv
    
    for i, t_i in enumerate(payment_times):
        t_prev = 0 if i == 0 else payment_times[i-1]
        dt = t_i - t_prev
        t_mid = (t_i + t_prev) / 2
        q_prev = survival_probability(t_prev, lambda_periods)
        q_i = survival_probability(t_i, lambda_periods)
        df_mid = np.exp(-r * t_mid)
        premium_leg += cds_rate * df_mid * (q_prev - q_i) * (dt / 2)
    
    protection_leg = 0.0
    for i, t_i in enumerate(payment_times):
        t_prev = 0 if i == 0 else payment_times[i-1]
        t_mid = (t_i + t_prev) / 2
        q_prev = survival_probability(t_prev, lambda_periods)
        q_i = survival_probability(t_i, lambda_periods)
        df_mid = np.exp(-r * t_mid)
        protection_leg += lgd * df_mid * (q_prev - q_i)
    
    return premium_leg - protection_leg


In [6]:
# Strip forward hazard rates –––––

dfFORWARD_LAMBDAS_Q2 = []
dfLAMBDA_PERIODS = []

for idx in range(len(vMATURITIES)):
    mat = vMATURITIES[idx]
    def objective(lam):
        temp_periods = dfLAMBDA_PERIODS.copy()
        t_start = 0 if idx == 0 else vMATURITIES[idx-1]
        temp_periods.append((t_start, mat, lam))
        return price_cds(mat, vCDS[idx], fLGD, fRISK_FREE, temp_periods)
    
    solution = brentq(objective, 0.0001, 1.0)
    dfFORWARD_LAMBDAS_Q2.append(solution)
    t_start = 0 if idx == 0 else vMATURITIES[idx-1]
    dfLAMBDA_PERIODS.append((t_start, mat, solution))


# Calculate average hazard rates and default probabilities
vAVG_HAZARD_RATES_Q2 = []
vDEFAULT_PROBS_Q2 = []
for idx in range(len(vMATURITIES)):
    mat = vMATURITIES[idx]
    integral = sum(lam * (min(t_end, mat) - t_start) 
                   for t_start, t_end, lam in dfLAMBDA_PERIODS if t_start < mat)
    vAVG_HAZARD_RATES_Q2.append(integral / mat)
    
    q_surv = survival_probability(mat, dfLAMBDA_PERIODS)
    q_prev = 1.0 if idx == 0 else survival_probability(vMATURITIES[idx-1], dfLAMBDA_PERIODS)
    vDEFAULT_PROBS_Q2.append(q_prev - q_surv)

# Compile results into DataFrame
dfRESULTS_Q2 = pd.DataFrame({
    'Maturity': [f'{m}Y' for m in vMATURITIES],
    'CDS Rate (bps)': vCDS_BPS,
    '(Average) Hazard Rate': vAVG_HAZARD_RATES_Q2,
    'Forward Hazard Rate': dfFORWARD_LAMBDAS_Q2,
    'Forward Default Probability': vDEFAULT_PROBS_Q2
})
print("\n", dfRESULTS_Q2.to_string(index=False))



 Maturity  CDS Rate (bps)  (Average) Hazard Rate  Forward Hazard Rate  Forward Default Probability
      1Y             100               0.024907             0.024907                     0.024599
      3Y             110               0.027472             0.028754                     0.054512
      5Y             120               0.030179             0.034239                     0.060950
      7Y             120               0.030096             0.029888                     0.049898
     10Y             125               0.031604             0.035124                     0.081013


In [7]:
# Export dfRESULTS_Q2 to LaTeX –––––
with open('Documentation/Tables/ccsr-a2-q2-results.tex', 'w') as f:
    f.write(dfRESULTS_Q2.to_latex(index=False, float_format="%.6f"))

## **Model Validation**

In [8]:
# Calculate 7Y CDS legs –––––

# Create payment times for 7-year CDS (quarterly payments)
vPAYMENT_TIMES = np.linspace(0.25, 7, int(7 * 4))

# Calculate premium leg and protection leg
premium_leg = sum(vCDS[3] * np.exp(-fRISK_FREE * t_i) * (t_i - (0 if i == 0 else vPAYMENT_TIMES[i-1])) * 
                  survival_probability(t_i, dfLAMBDA_PERIODS) 
                  for i, t_i in enumerate(vPAYMENT_TIMES))

# Add accrual payments for default between payment dates
premium_leg += sum(vCDS[3] * np.exp(-fRISK_FREE * (t_i + (0 if i == 0 else vPAYMENT_TIMES[i-1]))/2) * 
                   (survival_probability(0 if i == 0 else vPAYMENT_TIMES[i-1], dfLAMBDA_PERIODS) - 
                    survival_probability(t_i, dfLAMBDA_PERIODS)) * 
                   (t_i - (0 if i == 0 else vPAYMENT_TIMES[i-1])) / 2
                   for i, t_i in enumerate(vPAYMENT_TIMES))

# Protection leg
protection_leg = sum(fLGD * np.exp(-fRISK_FREE * (t_i + (0 if i == 0 else vPAYMENT_TIMES[i-1]))/2) * 
                     (survival_probability(0 if i == 0 else vPAYMENT_TIMES[i-1], dfLAMBDA_PERIODS) - 
                      survival_probability(t_i, dfLAMBDA_PERIODS))
                     for i, t_i in enumerate(vPAYMENT_TIMES))


# Display results
print(f"\nPremium Leg: {premium_leg:.10f}")
print(f"Protection Leg: {protection_leg:.10f}")
print(f"Difference: {abs(premium_leg - protection_leg):.2e}")
print(f"Result:{' Lower than tolerance' if abs(premium_leg - protection_leg) < 1e-6 else 'Failed'}")


Premium Leg: 0.0685500545
Protection Leg: 0.0685500545
Difference: 2.01e-14
Result: Lower than tolerance


## **Analysis & Stress Testing**

In [9]:
# Baseline comparison (r = 3%) –––––

print("Q4.1 Baseline Comparison (r = 3%)")

# Average hazard rates comparison
dfQ4_BASELINE_AVG = pd.DataFrame({
    "Maturity": [f"{m}Y" for m in vMATURITIES],
    "Q1_Avg_Hazard": vAVG_HAZARD_RATES_Q1,
    "Q2_Avg_Hazard": vAVG_HAZARD_RATES_Q2
})

dfQ4_BASELINE_AVG["Difference"] = (
    dfQ4_BASELINE_AVG["Q2_Avg_Hazard"] -
    dfQ4_BASELINE_AVG["Q1_Avg_Hazard"]
)

print("\nAverage Hazard Rates (Q1 vs Q2):")
print(dfQ4_BASELINE_AVG.to_string(index=False))


# Forward hazard rates comparison
labels = [
    f"0→{vMATURITIES[i]}Y" if i == 0
    else f"{vMATURITIES[i-1]}→{vMATURITIES[i]}Y"
    for i in range(len(vMATURITIES))
]

dfQ4_BASELINE_FWD = pd.DataFrame({
    "Period": labels,
    "Q1_Forward_Hazard": vFWD_HAZARD_RATES_Q1,
    "Q2_Forward_Hazard": dfFORWARD_LAMBDAS_Q2
})

dfQ4_BASELINE_FWD["Difference"] = (
    dfQ4_BASELINE_FWD["Q2_Forward_Hazard"] -
    dfQ4_BASELINE_FWD["Q1_Forward_Hazard"]
)

print("\nForward Hazard Rates (Q1 vs Q2):")
print(dfQ4_BASELINE_FWD.to_string(index=False))

Q4.1 Baseline Comparison (r = 3%)

Average Hazard Rates (Q1 vs Q2):
Maturity  Q1_Avg_Hazard  Q2_Avg_Hazard  Difference
      1Y        0.02500       0.024907   -0.000093
      3Y        0.02750       0.027472   -0.000028
      5Y        0.03000       0.030179    0.000179
      7Y        0.03000       0.030096    0.000096
     10Y        0.03125       0.031604    0.000354

Forward Hazard Rates (Q1 vs Q2):
Period  Q1_Forward_Hazard  Q2_Forward_Hazard  Difference
  0→1Y           0.025000           0.024907   -0.000093
  1→3Y           0.028750           0.028754    0.000004
  3→5Y           0.033750           0.034239    0.000489
  5→7Y           0.030000           0.029888   -0.000112
 7→10Y           0.034167           0.035124    0.000958


In [10]:
# Export dfQ4_BASELINE_AVG to LaTeX –––––
with open('Documentation/Tables/ccsr-a2-q41-baseline-avg.tex', 'w') as f:
    f.write(dfQ4_BASELINE_AVG.to_latex(index=False, float_format="%.6f"))

# Export dfQ4_BASELINE_FWD to LaTeX –––––
with open('Documentation/Tables/ccsr-a2-q41-baseline-fwd.tex', 'w') as f:
    f.write(dfQ4_BASELINE_FWD.to_latex(index=False, float_format="%.6f"))

In [11]:
# High Interest Rate Scenario (r = 10%) –––––

fRISK_FREE_HIGH = 0.10

print(f"High Interest Rate Scenario (r = {fRISK_FREE_HIGH:.0%})")

# Q1 under high rates (UNCHANGED)

# Q1 Average hazard rates (same formula)
vAVG_HAZARD_RATES_Q1_HIGH = vCDS / fLGD

# Q1 Forward hazard rates (same forward-difference logic)
vFWD_HAZARD_RATES_Q1_HIGH = np.zeros(len(vMATURITIES))
vFWD_HAZARD_RATES_Q1_HIGH[0] = vAVG_HAZARD_RATES_Q1_HIGH[0]
for i in range(1, len(vMATURITIES)):
    numerator = vAVG_HAZARD_RATES_Q1_HIGH[i] * vMATURITIES[i] - vAVG_HAZARD_RATES_Q1_HIGH[i-1] * vMATURITIES[i-1]
    vFWD_HAZARD_RATES_Q1_HIGH[i] = numerator / (vMATURITIES[i] - vMATURITIES[i-1])

# Q1 Default probabilities from exp(-λ_avg*T) (same as your Q1 method)
vSURVIVAL_PROBS_Q1_HIGH = np.exp(-vAVG_HAZARD_RATES_Q1_HIGH * vMATURITIES)

vDEFAULT_PROBS_Q1_HIGH = np.zeros(len(vMATURITIES))
vDEFAULT_PROBS_Q1_HIGH[0] = 1 - vSURVIVAL_PROBS_Q1_HIGH[0]
for i in range(1, len(vMATURITIES)):
    vDEFAULT_PROBS_Q1_HIGH[i] = vSURVIVAL_PROBS_Q1_HIGH[i-1] - vSURVIVAL_PROBS_Q1_HIGH[i]

dfRESULTS_Q1_HIGH = pd.DataFrame({
    "Maturity": [f"{m}Y" for m in vMATURITIES],
    "CDS Rate (bps)": vCDS_BPS,
    "(Average) Hazard Rate": vAVG_HAZARD_RATES_Q1_HIGH,
    "Forward Hazard Rate": vFWD_HAZARD_RATES_Q1_HIGH,
    "Default Probability": vDEFAULT_PROBS_Q1_HIGH
})

print("\nQ1 (unchanged):")
print(dfRESULTS_Q1_HIGH.to_string(index=False))


# Q2 under high rates (RE-STRIP with r = 10%)

dfFORWARD_LAMBDAS_Q2_HIGH = []
dfLAMBDA_PERIODS_HIGH = []

for idx in range(len(vMATURITIES)):
    mat = float(vMATURITIES[idx])

    def objective(lam):
        temp_periods = dfLAMBDA_PERIODS_HIGH.copy()
        t_start = 0.0 if idx == 0 else float(vMATURITIES[idx-1])
        temp_periods.append((t_start, mat, float(lam)))
        # IMPORTANT: only r changes here
        return price_cds(mat, float(vCDS[idx]), float(fLGD), float(fRISK_FREE_HIGH), temp_periods)

    sol = brentq(objective, 0.0001, 1.0)
    dfFORWARD_LAMBDAS_Q2_HIGH.append(float(sol))

    t_start = 0.0 if idx == 0 else float(vMATURITIES[idx-1])
    dfLAMBDA_PERIODS_HIGH.append((t_start, mat, float(sol)))


# Q2: compute average hazards + forward default probs from the stripped curve
vAVG_HAZARD_RATES_Q2_HIGH = []
vDEFAULT_PROBS_Q2_HIGH = []

for idx in range(len(vMATURITIES)):
    mat = float(vMATURITIES[idx])

    integral = sum(
        lam * (min(t_end, mat) - t_start)
        for t_start, t_end, lam in dfLAMBDA_PERIODS_HIGH
        if t_start < mat
    )
    vAVG_HAZARD_RATES_Q2_HIGH.append(float(integral / mat))

    q_surv = float(survival_probability(mat, dfLAMBDA_PERIODS_HIGH))
    q_prev = 1.0 if idx == 0 else float(survival_probability(float(vMATURITIES[idx-1]), dfLAMBDA_PERIODS_HIGH))
    vDEFAULT_PROBS_Q2_HIGH.append(float(q_prev - q_surv))

dfRESULTS_Q2_HIGH = pd.DataFrame({
    "Maturity": [f"{m}Y" for m in vMATURITIES],
    "CDS Rate (bps)": vCDS_BPS,
    "(Average) Hazard Rate": vAVG_HAZARD_RATES_Q2_HIGH,
    "Forward Hazard Rate": dfFORWARD_LAMBDAS_Q2_HIGH,
    "Forward Default Probability": vDEFAULT_PROBS_Q2_HIGH
})

print("\nQ2 (r = 10%):")
print(dfRESULTS_Q2_HIGH.to_string(index=False))


High Interest Rate Scenario (r = 10%)

Q1 (unchanged):
Maturity  CDS Rate (bps)  (Average) Hazard Rate  Forward Hazard Rate  Default Probability
      1Y             100                0.02500             0.025000             0.024690
      3Y             110                0.02750             0.028750             0.054498
      5Y             120                0.03000             0.033750             0.060103
      7Y             120                0.03000             0.030000             0.050124
     10Y             125                0.03125             0.034167             0.078969

Q2 (r = 10%):
Maturity  CDS Rate (bps)  (Average) Hazard Rate  Forward Hazard Rate  Forward Default Probability
      1Y             100               0.024690             0.024690                     0.024388
      3Y             110               0.027331             0.028651                     0.054333
      5Y             120               0.030311             0.034782                     0.06190

In [12]:
# Export dfRESULTS_Q1_HIGH to LaTeX –––––
with open('Documentation/Tables/ccsr-a2-q42-high-rate-results-q1.tex', 'w') as f:
    f.write(dfRESULTS_Q1_HIGH.to_latex(index=False, float_format="%.6f"))

# Export dfRESULTS_Q2_HIGH to LaTeX –––––
with open('Documentation/Tables/ccsr-a2-q42-high-rate-results.tex', 'w') as f:
    f.write(dfRESULTS_Q2_HIGH.to_latex(index=False, float_format="%.6f"))


In [13]:
# Re-strip Q2 at r = 10% + build vectors + build DataFrames –––––

# High interest rate
fRISK_FREE_HIGH = 0.10

# Strip forward hazard rates for Q2 under r = 10%
dfFORWARD_LAMBDAS_Q2_HIGH = []
dfLAMBDA_PERIODS_HIGH = []

for idx in range(len(vMATURITIES)):
    mat = float(vMATURITIES[idx])

    def objective(lam):
        temp_periods = dfLAMBDA_PERIODS_HIGH.copy()
        t_start = 0.0 if idx == 0 else float(vMATURITIES[idx - 1])
        temp_periods.append((t_start, mat, float(lam)))
        return price_cds(mat, float(vCDS[idx]), float(fLGD), float(fRISK_FREE_HIGH), temp_periods)

    sol = brentq(objective, 0.0001, 1.0)
    dfFORWARD_LAMBDAS_Q2_HIGH.append(float(sol))

    t_start = 0.0 if idx == 0 else float(vMATURITIES[idx - 1])
    dfLAMBDA_PERIODS_HIGH.append((t_start, mat, float(sol)))


# Compute average hazard rates and forward default probabilities implied by dfLAMBDA_PERIODS_HIGH
vAVG_HAZARD_RATES_Q2_HIGH = []
vDEFAULT_PROBS_Q2_HIGH = []  # forward (period) default probs: Q(t_{i-1}) - Q(t_i)

for idx in range(len(vMATURITIES)):
    mat = float(vMATURITIES[idx])

    # average hazard over [0, mat] = (∫0^mat λ(u)du) / mat
    integral = sum(
        lam * (min(t_end, mat) - t_start)
        for t_start, t_end, lam in dfLAMBDA_PERIODS_HIGH
        if t_start < mat
    )
    vAVG_HAZARD_RATES_Q2_HIGH.append(float(integral / mat))

    # forward default probability over the maturity bucket
    q_surv = float(survival_probability(mat, dfLAMBDA_PERIODS_HIGH))
    q_prev = 1.0 if idx == 0 else float(survival_probability(float(vMATURITIES[idx - 1]), dfLAMBDA_PERIODS_HIGH))
    vDEFAULT_PROBS_Q2_HIGH.append(float(q_prev - q_surv))


# Build DataFrames (labels for forward periods)
labels = [
    f"0→{vMATURITIES[i]}Y" if i == 0 else f"{vMATURITIES[i-1]}→{vMATURITIES[i]}Y"
    for i in range(len(vMATURITIES))
]


In [14]:
# Store Q2 high-rate results
dfQ4_Q2_HIGH = pd.DataFrame({
    "Maturity": [f"{m}Y" for m in vMATURITIES],
    "Q2_Avg_Hazard_3pct": vAVG_HAZARD_RATES_Q2,
    "Q2_Avg_Hazard_10pct": vAVG_HAZARD_RATES_Q2_HIGH
})

dfQ4_Q2_HIGH["Difference"] = (
    dfQ4_Q2_HIGH["Q2_Avg_Hazard_10pct"] -
    dfQ4_Q2_HIGH["Q2_Avg_Hazard_3pct"]
)

print("\nQ2 Average Hazard Rates: 3% vs 10%")
print(dfQ4_Q2_HIGH.to_string(index=False))


# Forward hazard comparison
dfQ4_Q2_FWD_HIGH = pd.DataFrame({
    "Period": labels,
    "Q2_Forward_3pct": dfFORWARD_LAMBDAS_Q2,
    "Q2_Forward_10pct": dfFORWARD_LAMBDAS_Q2_HIGH
})

dfQ4_Q2_FWD_HIGH["Difference"] = (
    dfQ4_Q2_FWD_HIGH["Q2_Forward_10pct"] -
    dfQ4_Q2_FWD_HIGH["Q2_Forward_3pct"]
)

print("\nQ2 Forward Hazard Rates: 3% vs 10%")
print(dfQ4_Q2_FWD_HIGH.to_string(index=False))



Q2 Average Hazard Rates: 3% vs 10%
Maturity  Q2_Avg_Hazard_3pct  Q2_Avg_Hazard_10pct  Difference
      1Y            0.024907             0.024690   -0.000216
      3Y            0.027472             0.027331   -0.000141
      5Y            0.030179             0.030311    0.000132
      7Y            0.030096             0.030116    0.000020
     10Y            0.031604             0.032066    0.000461

Q2 Forward Hazard Rates: 3% vs 10%
Period  Q2_Forward_3pct  Q2_Forward_10pct  Difference
  0→1Y         0.024907          0.024690   -0.000216
  1→3Y         0.028754          0.028651   -0.000103
  3→5Y         0.034239          0.034782    0.000543
  5→7Y         0.029888          0.029629   -0.000259
 7→10Y         0.035124          0.036615    0.001491


In [15]:
# Export dfQ4_Q2_HIGH to LaTeX –––––
with open('Documentation/Tables/ccsr-a2-q42-high-rate-avg.tex', 'w') as f:
    f.write(dfQ4_Q2_HIGH.to_latex(index=False, float_format="%.6f"))

# Export dfQ4_Q2_FWD_HIGH to LaTeX –––––
with open('Documentation/Tables/ccsr-a2-q42-high-rate-fwd.tex', 'w') as f:
    f.write(dfQ4_Q2_FWD_HIGH.to_latex(index=False, float_format="%.6f"))

## **Code export**

In [16]:
# Export python code / markdown cells separately –––––

# Config –
NAME = "ccsr-a2"

# Import –
import nbformat

# Load notebook –
nb = nbformat.read(f"{NAME}-script.ipynb", as_version=4)

# Write code cells into one python script –
with open(f"Documentation/{NAME}-script.py", "w", encoding="utf-8") as f:
    for cell in nb.cells:
        if cell.cell_type == "code":
            f.write(cell.source.replace("–", "-") + "\n\n")


# Collect markdown cells –
md_cells = [cell['source'] for cell in nb.cells if cell['cell_type'] == 'markdown']

# Write them into one markdown file
with open(f"Documentation/{NAME}-markdown.md", "w") as f:
    f.write("\n\n".join(md_cells))


# Convert markdown to LaTeX using pandoc –
# rewrite and run in bash
# pandoc Documentation/{NAME}-markdown.md -o Documentation/{NAME}-markdown.tex